In [ ]:
%%capture

!pip install -U pip transformers
# !pip install emoji --upgrade

In [ ]:
# ----------------------------- IMPORTS ------------------------------ #

# -------------------------------------------------------------------- #
# CSV AND DF MANIPULATION
import pandas as pd
import numpy as np
import csv

# -------------------------------------------------------------------- #
# STRING AND REGEX MANIPULATION
import string
# import emoji
import re
import os

# -------------------------------------------------------------------- #
# TRANSLATION MODELS
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
                                          # ^  HUGGINGFACE (NLLB, mBART,
                                          # |______________ OPUS)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = #Your path

Mounted at /content/drive


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-es-en"

In [ ]:
%%capture

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
%%capture
detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
text = 'Yacouba Sylla Aston Villa ) explique sa première sélection avec les Aigles du Mali	'
result = detector(text)
print(result[0]['label'])  # Output: 'it'


fr


In [ ]:
text = '''
Pence q la novia de Messi lo mordia a josema le tendria q haber partido una pata a ese alcaguete pesero de mierda no me jodan mas con SUAREZ
'''

In [ ]:
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

I thought Messi's girlfriend bit him to josema I'd have to kick the shit out of that shitty puss man. Don't fuck me with SUAREZ anymore.


In [ ]:
len(os.listdir(os.path.join(path, 'youtube-database-3')))

2288

In [ ]:
for file in ['youtube-database-1', 'youtube-database-2', 'youtube-database-3']:
  print(len(os.listdir(os.path.join(path, file))))

2288
2288
2288


## Test individual de traducción de archivo `csv`





### Detección de idioma

In [ ]:
def detect_lang(text):
  if isinstance(text, str) and text.strip():
    while True:
      if len(text):
        try:
          result = detector(text)
          break
        except:
          text = " ".join(text.split()[:-1])
      else:
        return None
    return result[0]['label']
  else:
    return None

In [ ]:
%%capture
detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")
new_path = os.path.join(path, 'youtube-database-3')

file_count = 0

for file in os.listdir(new_path):
  file_path = os.path.join(new_path, file)
  print(file.split('-')[0])
  df_file = pd.read_csv(file_path)
  if df_file.size and 'lang' not in df_file.columns:
    try:
      df_file['lang'] = df_file['comment'].apply(detect_lang)
      print('Comment file labeled')
    except KeyError:
      df_file['lang'] = df_file['title'].apply(detect_lang)
      print('Title file labeled')
    df_file.to_csv(file_path, index=False)
    print(f'File {file} replaced')
    file_count += 1
    print(f'{file_count} files checked')
  else:
    print('Already labeled language')
    file_count += 1
    print(f'{file_count} files checked')
    continue

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Se han truncado las últimas 5000 líneas del flujo de salida.
177634
Already labeled language
702 files checked
177634
Already labeled language
703 files checked
199353
Already labeled language
704 files checked
199353
Already labeled language
705 files checked
204024
Already labeled language
706 files checked
204024
Already labeled language
707 files checked
208120
Already labeled language
708 files checked
208120
Already labeled language
709 files checked
203838
Already labeled language
710 files checked
212463
Already labeled language
711 files checked
212463
Already labeled language
712 files checked
189389
Already labeled language
713 files checked
189389
Already labeled language
714 files checked
190824
Already labeled language
715 files checked
190824
Already labeled language
716 files checked
183662
Already labeled language
717 files checked
204637
Already labeled language
718 files checked
183662
Already labeled language
719 files checked
204637
Already labeled language
720 fil

### Construcción de modelos de traducción

In [ ]:
%%capture

model_name_es = f"Helsinki-NLP/opus-mt-es-en"
tokenizer_es = AutoTokenizer.from_pretrained(model_name_es)
model_es = AutoModelForSeq2SeqLM.from_pretrained(model_name_es)

model_name_de = f"Helsinki-NLP/opus-mt-de-en"
tokenizer_de = AutoTokenizer.from_pretrained(model_name_de)
model_de = AutoModelForSeq2SeqLM.from_pretrained(model_name_de)

model_name_fr = f"Helsinki-NLP/opus-mt-fr-en"
tokenizer_fr = AutoTokenizer.from_pretrained(model_name_fr)
model_fr = AutoModelForSeq2SeqLM.from_pretrained(model_name_fr)

model_name_it = f"Helsinki-NLP/opus-mt-it-en"
tokenizer_it = AutoTokenizer.from_pretrained(model_name_it)
model_it = AutoModelForSeq2SeqLM.from_pretrained(model_name_it)

### Función de traducción de texto

In [ ]:
def translate_text(text, language):
  if language == 'en':
    return text

  elif language == 'es':
    tokenizer = tokenizer_es
    model = model_es

  elif language == 'de':
    tokenizer = tokenizer_de
    model = model_de

  elif language == 'fr':
    tokenizer = tokenizer_fr
    model = model_fr

  elif language == 'it':
    tokenizer = tokenizer_it
    model = model_it

  else:
    return None

  while True:
    try:
      inputs = tokenizer(text, return_tensors="pt")
      outputs = model.generate(**inputs)
      output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
      break
    except:
      text = " ".join(text.split()[:-1])
  return output_text

def apply_translation(df, column_name):
  rows_translated = 0
  rows_to_translate = len(df)

  def translation_with_counter(row):
    nonlocal rows_translated
    rows_translated += 1
    if rows_translated == int(rows_to_translate*0.25):
      print(f'Translation 25% done: {rows_translated}/{rows_to_translate} translated')
    elif rows_translated == int(rows_to_translate*0.5):
      print(f'Translation 50% done: {rows_translated}/{rows_to_translate} translated')
    elif rows_translated == int(rows_to_translate*0.75):
      print(f'Translation 75% done: {rows_translated}/{rows_to_translate} translated')

    return translate_text(row[column_name], row['lang'])

  print(f'Translating {rows_to_translate}...')
  df['translation_en'] = df.apply(translation_with_counter, axis=1)
  print(f'Translation completed: {rows_translated}/{rows_to_translate} translated')

### Traducción de archivos

In [ ]:
# new_path = os.path.join(path, 'youtube-database-1')
# new_path = os.path.join(path, 'youtube-database-2')
new_path = os.path.join(path, 'youtube-database-3')

file_count = 0

for file in os.listdir(new_path):
  file_path = os.path.join(new_path, file)
  print(file.split('-')[0])
  df_file = pd.read_csv(file_path)

  if df_file.size and 'translation_en' not in df_file.columns:
    texts_count = 0
    texts_to_translate = df_file.shape[0]
    try:
      # df_file['translation_en'] = df_file.apply(lambda row: translate_text(row['comment'], row['lang']), axis=1)
      apply_translation(df_file, 'comment')
      print('Comment file labeled')
    except KeyError:
      # df_file['translation_en'] = df_file.apply(lambda row: translate_text(row['title'], row['lang']), axis=1)
      apply_translation(df_file, 'title')
      print('Title file labeled')

    df_file.to_csv(file_path, index=False)
    print(f'File {file} replaced')
    file_count += 1
    print(f'{file_count} files checked')

  else:
    print('Already translated file')
    file_count += 1
    print(f'{file_count} files checked')
    continue

193235
Already translated file
1 files checked
203980
Already translated file
2 files checked
203980
Already translated file
3 files checked
194092
Already translated file
4 files checked
194092
Already translated file
5 files checked
203161
Already translated file
6 files checked
204494
Already translated file
7 files checked
203161
Already translated file
8 files checked
204494
Already translated file
9 files checked
187818
Already translated file
10 files checked
187818
Already translated file
11 files checked
201860
Already translated file
12 files checked
199780
Already translated file
13 files checked
199780
Already translated file
14 files checked
201860
Already translated file
15 files checked
208702
Already translated file
16 files checked
206606
Already translated file
17 files checked
206606
Already translated file
18 files checked
210021
Already translated file
19 files checked
210021
Already translated file
20 files checked
193896
Already translated file
21 files checked
1

Token indices sequence length is longer than the specified maximum sequence length for this model (782 > 512). Running this sequence through the model will result in indexing errors


Translation 25% done: 39/157 translated
Translation 50% done: 78/157 translated
Translation 75% done: 117/157 translated
Translation completed: 157/157 translated
Comment file labeled
File 205632-comments-3.csv replaced
1032 files checked
213388
Translating 22...
Translating 22...
Translation 25% done: 5/22 translated
Translation 50% done: 11/22 translated
Translation 75% done: 16/22 translated
Translation completed: 22/22 translated
Title file labeled
File 213388-titles-3.csv replaced
1033 files checked
213388
Translating 54...
Translation 25% done: 13/54 translated
Translation 50% done: 27/54 translated
Translation 75% done: 40/54 translated
Translation completed: 54/54 translated
Comment file labeled
File 213388-comments-3.csv replaced
1034 files checked
190685
Translating 191...
Translation 25% done: 47/191 translated
Translation 50% done: 95/191 translated
Translation 75% done: 143/191 translated
Translation completed: 191/191 translated
Comment file labeled
File 190685-comments-3

Token indices sequence length is longer than the specified maximum sequence length for this model (677 > 512). Running this sequence through the model will result in indexing errors


Se han truncado las últimas 5000 líneas del flujo de salida.
Title file labeled
File 201417-titles-3.csv replaced
1362 files checked
201417
Translating 18...
Translation 25% done: 4/18 translated
Translation 50% done: 9/18 translated
Translation 75% done: 13/18 translated
Translation completed: 18/18 translated
Comment file labeled
File 201417-comments-3.csv replaced
1363 files checked
178628
Translating 50...
Translating 50...
Translation 25% done: 12/50 translated
Translation 50% done: 25/50 translated
Translation 75% done: 37/50 translated
Translation completed: 50/50 translated
Title file labeled
File 178628-titles-3.csv replaced
1364 files checked
169710
Translating 15...
Translating 15...
Translation 25% done: 3/15 translated
Translation 50% done: 7/15 translated
Translation 75% done: 11/15 translated
Translation completed: 15/15 translated
Title file labeled
File 169710-titles-3.csv replaced
1365 files checked
169710
Translating 9...
Translation 25% done: 2/9 translated
Translat